In [89]:
import text2term
import pandas
import pickle
import os
import pandas as pd
import json
from datetime import datetime
import time
import random

In [2]:
text2term.cache_ontology("https://purl.obolibrary.org/obo/ncbitaxon.owl", "ncbitaxon")

2023-11-11 08:07:54 INFO [text2term.term_collector]: Loading ontology https://purl.obolibrary.org/obo/ncbitaxon.owl...
2023-11-11 08:21:48 INFO [text2term.term_collector]: ...done (ontology loading time: 589.27s)
2023-11-11 08:21:48 INFO [text2term.term_collector]: Collecting ontology term details...
2023-11-11 08:28:29 INFO [text2term.term_collector]: ...done: collected 2523789 ontology terms (collection time: 401.07s)
2023-11-11 08:33:36 INFO [text2term.t2t]: Filtered ontology terms to those of type: any
2023-11-11 08:33:37 INFO [text2term.t2t]: Caching ontology https://purl.obolibrary.org/obo/ncbitaxon.owl to: cache\ncbitaxon


In [8]:
with open(os.path.join('data','raw_species_results.pickle'),'rb') as infile:
    raw_species = pickle.load(infile)

print(raw_species.head(n=2))

             _id       _ignored  _score  \
0  GEO_GSE110840  [all.keyword]     1.0   
1  GEO_GSE110842  [all.keyword]     1.0   

                                                name  \
0  Sequencing of Caenorhabditis elegans deletion ...   
1  Sequencing of Caenorhabditis elegans overexpre...   

                                species  
0  [{'name': 'Caenorhabditis elegans'}]  
1  [{'name': 'Caenorhabditis elegans'}]  


In [15]:
print(len(raw_species))

raw_boom = raw_species.explode('species')
print(len(raw_boom))
print(raw_boom.head(n=2))

200837
202257
             _id       _ignored  _score  \
0  GEO_GSE110840  [all.keyword]     1.0   
1  GEO_GSE110842  [all.keyword]     1.0   

                                                name  \
0  Sequencing of Caenorhabditis elegans deletion ...   
1  Sequencing of Caenorhabditis elegans overexpre...   

                              species  
0  {'name': 'Caenorhabditis elegans'}  
1  {'name': 'Caenorhabditis elegans'}  


In [24]:
def pop_species(species_dict):
    if isinstance(species_dict,dict):
        x = species_dict['name']
    elif isinstance(species_dict,str):
        x = species_dict
    return x

## Raise a flag if the species was not formatted as a dictionary
def flag_species(species_dict):
    if isinstance(species_dict,dict):
        x = False
    elif isinstance(species_dict,str):
        x = True
    return x    

raw_boom['species_name'] = raw_boom.apply(lambda row: pop_species(row['species']),axis=1)
raw_boom['flag_raised'] = raw_boom.apply(lambda row: flag_species(row['species']),axis=1)
print(raw_boom.head(n=2))
clean_dict = raw_boom[['_id','name','species_name','flag_raised']].copy()

             _id       _ignored  _score  \
0  GEO_GSE110840  [all.keyword]     1.0   
1  GEO_GSE110842  [all.keyword]     1.0   

                                                name  \
0  Sequencing of Caenorhabditis elegans deletion ...   
1  Sequencing of Caenorhabditis elegans overexpre...   

                              species            species_name  flag_raised  
0  {'name': 'Caenorhabditis elegans'}  Caenorhabditis elegans        False  
1  {'name': 'Caenorhabditis elegans'}  Caenorhabditis elegans        False  


In [25]:
print(clean_dict.head(n=2))

             _id                                               name  \
0  GEO_GSE110840  Sequencing of Caenorhabditis elegans deletion ...   
1  GEO_GSE110842  Sequencing of Caenorhabditis elegans overexpre...   

             species_name  flag_raised  
0  Caenorhabditis elegans        False  
1  Caenorhabditis elegans        False  


In [27]:
no_dups = clean_dict['species_name'].unique().tolist()
print(len(no_dups))

2565


In [28]:
%%time
t2t_result = text2term.map_terms(no_dups, "ncbitaxon", use_cache=True)

2023-11-13 10:38:58 INFO [text2term.t2t]: Loading cached ontology from: cache\ncbitaxon\ncbitaxon-term-details.pickle
2023-11-13 10:41:30 INFO [text2term.t2t]: Filtered ontology terms to those of type: class
2023-11-13 10:41:30 INFO [text2term.t2t]: Mapping 2565 source terms to ncbitaxon
2023-11-13 10:46:52 INFO [text2term.t2t]: ...done (mapping time: 321.58s seconds)
CPU times: total: 7min 57s
Wall time: 8min 33s


In [31]:
t2t_result.sort_values(['Source Term','Mapping Score'], ascending = [True,False], inplace=True)
print(t2t_result.head(n=2))

                                  Source Term ID    Source Term  \
3180  http://ccb.hms.harvard.edu/t2t/R3Go4L6RwRz  Abies pinsapo   
3181  http://ccb.hms.harvard.edu/t2t/R3Go4L6RwRz  Abies pinsapo   

               Mapped Term Label Mapped Term CURIE  \
3180               Abies pinsapo   NCBITAXON:56046   
3181  Abies pinsapo var. pinsapo  NCBITAXON:928732   

                                      Mapped Term IRI  Mapping Score  Tags  
3180   http://purl.obolibrary.org/obo/NCBITaxon_56046          0.997  None  
3181  http://purl.obolibrary.org/obo/NCBITaxon_928732          0.923  None  


In [32]:
top_score = t2t_result.drop_duplicates('Source Term',keep='first')
print(len(top_score))

2564


In [34]:
t2t_dictdf = top_score[['Source Term','Mapped Term Label','Mapped Term CURIE', 'Mapping Score']].copy()
t2t_dictdf.rename(columns={'Source Term':'species_name', 'Mapped Term CURIE':'CURIE','Mapped Term Label':'Label'}, inplace=True)

In [36]:
mapped_dict = clean_dict.merge(t2t_dictdf, on='species_name', how='inner')
print(len(mapped_dict))
mapped_dict.drop_duplicates(keep='first', inplace=True)
print(len(mapped_dict))

202256
202182


In [41]:
no_flags = mapped_dict.loc[mapped_dict['flag_raised']==False]
flagged = mapped_dict.loc[mapped_dict['flag_raised']==True]
print(len(no_flags))

197509


In [43]:
no_flags.to_csv(os.path.join('data','t2t_mapped_no_flags.tsv'), sep='\t', header=True)
flagged.to_csv(os.path.join('data','t2t_mapped_flagged.tsv'), sep='\t', header=True)

## Process the data from the EXTRACT test

In [59]:
## set filepaths
script_path = os.getcwd()
parent_path = os.path.dirname(script_path)
input_path = os.path.join(parent_path,'EXTRACT_check','data')
input_file = os.path.join(input_path,'test_100000.tsv')
output_path = os.path.join(script_path,'data')
stats = {}

In [60]:
extract_results = pd.read_csv(input_file,delimiter='\t',header=0,index_col=0)
print(len(extract_results))
print(extract_results.head(n=2))
stats['number of results from EXTRACT']=len(extract_results)

210242
         _id extracted_term  extracted_type  taxid           CURIE
0  GSE238109         bovine              -2   9913  NCBITAXON:9913
1  GSE187829          human              -2   9606  NCBITAXON:9606


In [66]:
print(len(extract_results.loc[(extract_results['extracted_type']!=-2)]))

0


In [4]:
unique_terms = extract_results['extracted_term'].unique().tolist()
print(len(unique_terms))
print(unique_terms[0:10])
stats['number of unique extracted terms']=len(unique_terms)

8851
['bovine', 'human', 'mouse', 'mice', 'murine', 'Human', 'P. gingivalis', 'Porphyromonas gingivalis', 'Mouse', 'MCMV']


### Check if casing matters in processing of terms

In [21]:
## Note capitalization introduces duplication; however, proper casing can be useful in species determination
## We'll also test whether or not capitalization affects Text2Term mappings
lower_unique = list(set([x.lower() for x in unique_terms]))
print(len(lower_unique))
print(lower_unique[0:10])

## The lower casing and de-duplication removed only about 12% of the terms
stats['number of unique terms if casing adjusted (to lower case)']= len(lower_unique)
stats['percent change if casing adjusted'] = (len(unique_terms)-len(lower_unique)/len(unique_terms))

with open(os.path.join(output_path,'t2t_EXTRACT_test.json'),'w') as outwrite:
    outwrite.write(json.dumps(stats))

7727
['artemisia afra', 'p. yoelii nigeriensis', 'pyrus pyrifolia', 'lined ground squirrels', 'anabaena pcc7120', 'potato psyllid', 'caenorhabditis elegans', 'anopheles moucheti', 'b. germanica', 'b. bacteriovorus']


TypeError: Object of type datetime is not JSON serializable

In [16]:
uniquestarttime = datetime.now()
time.sleep(1)
uniqueendtime = datetime.now()
duration = uniqueendtime-uniquestarttime
print(duration)

with open(os.path.join(output_path,'t2t_EXTRACT_test.json'),'r') as statfile:
    stats = json.loads(statfile.read())
    print(stats)

0:00:01.016336
{'number of results from EXTRACT': 210242, 'number of unique extracted terms': 8851, 'number of unique terms if casing adjusted (to lower case)': 7727, 'percent change if casing adjusted': 8850.126991300418}


In [17]:
%%time
with open(os.path.join(output_path,'t2t_EXTRACT_test.json'),'r') as statfile:
    stats = json.loads(statfile.read())
    print(stats)
uniquestarttime = datetime.now()
stats['Time start for T2T test on unique terms (casing not adjusted)'] = datetime.strftime(uniquestarttime,'%m/%d/%Y, %H:%M:%S')

unique_t2t_result = text2term.map_terms(unique_terms, "ncbitaxon", use_cache=True)

uniqueendtime = datetime.now()
stats['Time end for T2T test on unique terms (casing not adjusted)'] = datetime.strftime(uniqueendtime,'%m/%d/%Y, %H:%M:%S')
stats['Duration of T2T test on unique terms (casing not adjusted)'] = str(uniqueendtime - uniquestarttime)

with open(os.path.join(output_path,'EXTRACT_unique_t2t_test.pickle'),'wb') as outfile:
    pickle.dump(unique_t2t_result,outfile)

with open(os.path.join(output_path,'t2t_EXTRACT_test.json'),'w') as outwrite:
    outwrite.write(json.dumps(stats))

{'number of results from EXTRACT': 210242, 'number of unique extracted terms': 8851, 'number of unique terms if casing adjusted (to lower case)': 7727, 'percent change if casing adjusted': 8850.126991300418}
2023-11-18 08:49:02 INFO [text2term.t2t]: Loading cached ontology from: cache\ncbitaxon\ncbitaxon-term-details.pickle
2023-11-18 08:49:44 INFO [text2term.t2t]: Filtered ontology terms to those of type: class
2023-11-18 08:49:44 INFO [text2term.t2t]: Mapping 8851 source terms to ncbitaxon
2023-11-18 08:54:48 INFO [text2term.t2t]: ...done (mapping time: 303.78s seconds)


NameError: name 'query_type' is not defined

In [25]:
%%time
with open(os.path.join(output_path,'t2t_EXTRACT_test.json'),'r') as statfile:
    stats = json.loads(statfile.read())
    print(stats)

lowerstarttime = datetime.now()
stats['Time start for T2T test on unique terms (casing adjusted)'] = datetime.strftime(lowerstarttime,'%m/%d/%Y, %H:%M:%S')

lower_t2t_result = text2term.map_terms(lower_unique, "ncbitaxon", use_cache=True)

lowerendtime = datetime.now()
stats['Time end for T2T test on unique terms (casing adjusted)'] = datetime.strftime(lowerendtime,'%m/%d/%Y, %H:%M:%S')
stats['Duration of T2T test on unique terms (casing adjusted)'] = str(lowerendtime - lowerstarttime)

with open(os.path.join(output_path,'EXTRACT_lower_t2t_test.pickle'),'wb') as loweroutfile:
    pickle.dump(lower_t2t_result,loweroutfile)  

with open(os.path.join(output_path,'t2t_EXTRACT_test.json'),'w') as outwrite:
    outwrite.write(json.dumps(stats))

{'number of results from EXTRACT': 210242, 'number of unique extracted terms': 8851, 'number of unique terms if casing adjusted (to lower case)': 7727, 'percent change if casing adjusted': 8850.126991300418, 'Time start for T2T test on unique terms (casing not adjusted)': '11/18/2023, 08:49:02', 'Time end for T2T test on unique terms (casing not adjusted)': '11/18/2023, 08:57:08', 'Duration of T2T test on unique terms (casing not adjusted)': '0:08:06.170061'}
2023-11-18 10:16:59 INFO [text2term.t2t]: Loading cached ontology from: cache\ncbitaxon\ncbitaxon-term-details.pickle
2023-11-18 10:17:54 INFO [text2term.t2t]: Filtered ontology terms to those of type: class
2023-11-18 10:17:54 INFO [text2term.t2t]: Mapping 7727 source terms to ncbitaxon
2023-11-18 10:23:27 INFO [text2term.t2t]: ...done (mapping time: 332.62s seconds)
CPU times: total: 8min 34s
Wall time: 9min 2s


#### Actually run the comparison

In [39]:
## Compare processing of case-adjusted vs original case mappings
with open(os.path.join(output_path,'EXTRACT_lower_t2t_test.pickle'),'rb') as loweroutfile:
    lower_t2t_result = pickle.load(loweroutfile)  

with open(os.path.join(output_path,'EXTRACT_unique_t2t_test.pickle'),'rb') as outfile:
    unique_t2t_result = pickle.load(outfile) 
    
## rename columns and add lower case column in preparation for a merge
unique_t2t_result.rename(columns={'Source Term':'original mapped term','Mapping Score':'Original Mapping Score'}, inplace=True)
unique_t2t_result['Source Term'] = [x.lower() for x in unique_t2t_result['original mapped term']]

lower_t2t_result.rename(columns={'Mapping Score':'Lower Case Map Score'},inplace = True)

print(unique_t2t_result.head(n=2))
print(lower_t2t_result.head(n=2))

                               Source Term ID original mapped term  \
0  http://ccb.hms.harvard.edu/t2t/RKMJbF6HJke               bovine   
1  http://ccb.hms.harvard.edu/t2t/RKMJbF6HJke               bovine   

  Mapped Term Label  Mapped Term CURIE  \
0        Bos taurus     NCBITAXON:9913   
1  Bovine nebovirus  NCBITAXON:1661258   

                                    Mapped Term IRI  Original Mapping Score  \
0     http://purl.obolibrary.org/obo/NCBITaxon_9913                   0.986   
1  http://purl.obolibrary.org/obo/NCBITaxon_1661258                   0.772   

   Tags Source Term  
0  None      bovine  
1  None      bovine  
                               Source Term ID     Source Term  \
0  http://ccb.hms.harvard.edu/t2t/RKFFMZsif39  artemisia afra   
1  http://ccb.hms.harvard.edu/t2t/RKFFMZsif39  artemisia afra   

  Mapped Term Label Mapped Term CURIE  \
0    Artemisia afra   NCBITAXON:72333   
1         Artemisia    NCBITAXON:4219   

                                  Mapp

In [53]:
unique2merge = unique_t2t_result[['original mapped term','Source Term','Mapped Term Label','Mapped Term CURIE','Original Mapping Score']].copy()
unique2merge.sort_values(['Source Term','Original Mapping Score'], ascending=[True,False],inplace=True)
unique2merge.drop_duplicates(subset=['original mapped term','Source Term'],keep='first',inplace=True)

lower2merge = lower_t2t_result[['Source Term','Mapped Term Label','Mapped Term CURIE','Lower Case Map Score']].copy()
lower2merge.sort_values(['Source Term','Lower Case Map Score'], ascending=[True,False],inplace=True)
lower2merge.drop_duplicates(subset=['Source Term'],keep='first',inplace=True)
print(len(unique2merge))
print(len(lower2merge))

mergeddf = unique2merge.merge(lower2merge,on=['Source Term','Mapped Term Label','Mapped Term CURIE'],how='left')
matchedmergeddf = unique2merge.merge(lower2merge,on=['Source Term','Mapped Term Label','Mapped Term CURIE'],how='inner')
print(len(matchedmergeddf))

lower_map_fail = mergeddf.loc[mergeddf['Lower Case Map Score'].isna()]
print(lower_map_fail.head(n=4))

## inspect some of the failures to see if any improvements can be made

8844
7720
8678
    original mapped term   Source Term   Mapped Term Label  Mapped Term CURIE  \
71            A. linaria    a. linaria     Linaria <birds>  NCBITAXON:1930290   
108         A. stephensi  a. stephensi  Stephensia <moths>  NCBITAXON:1073692   
134                 abaM          abam       Musa textilis   NCBITAXON:228183   
246                Aedes         aedes    Aedes <subgenus>   NCBITAXON:149531   

     Original Mapping Score  Lower Case Map Score  
71                    0.938                   NaN  
108                   0.823                   NaN  
134                   0.495                   NaN  
246                   0.999                   NaN  


In [68]:
## Check if score for terms which were mapped twice are higher or lower based on casing
original_higher = matchedmergeddf.loc[matchedmergeddf['Original Mapping Score']>matchedmergeddf['Lower Case Map Score']]
lower_case_higher = matchedmergeddf.loc[matchedmergeddf['Original Mapping Score']<matchedmergeddf['Lower Case Map Score']]
scores_equal = matchedmergeddf.loc[matchedmergeddf['Original Mapping Score']==matchedmergeddf['Lower Case Map Score']]
print(len(original_higher))
print(len(lower_case_higher))
print(len(scores_equal))

print(original_higher.head(n=4))
print('=================')
print(lower_case_higher.head(n=4))
print('=================')
print(scores_equal.head(n=4))

1227
2877
4574
              original mapped term                      Source Term  \
2                            5z489                            5z489   
3                           A- RNA                           a- rna   
4                            A-RNA                            a-rna   
9  A. actinomycetemcomitans HK1651  a. actinomycetemcomitans hk1651   

                              Mapped Term Label  Mapped Term CURIE  \
2                              Enytus sp. Oz489  NCBITAXON:2071065   
3                                 RNA virus sp.  NCBITAXON:2217650   
4                                 RNA virus sp.  NCBITAXON:2217650   
9  Aggregatibacter actinomycetemcomitans HK1651   NCBITAXON:272556   

   Original Mapping Score  Lower Case Map Score  
2                   0.448                 0.447  
3                   0.756                 0.755  
4                   0.756                 0.755  
9                   0.816                 0.815  
  original mapped term    So

lower casing does not save too much time, so preserve it in order to ensure good capture of species in the [Letter. word] format

In some cases, T2T will give a match to the species term a higher score even though it's in the wrong genus. To select the right match, it may be necessary to pull these cases out by regex and select the match that includes the correct genus abbreviation


In [49]:
term_w_periods = [x for x in unique2merge['original mapped term'] if '.' in x]
print(term_w_periods[0:10])
print(len(term_w_periods))

['A.  fumigatus', 'A. aceti', 'A. actinomycetemcomitans', 'A. actinomycetemcomitans HK1651', 'A. aculeatus', 'A. adenophora', 'A. aegypti', 'A. afra', 'A. afraspera', 'A. alternata']
1781


In [50]:
t2t_period = text2term.map_terms(term_w_periods, "ncbitaxon", use_cache=True)
print(len(t2t_period))

2023-11-18 11:02:14 INFO [text2term.t2t]: Loading cached ontology from: cache\ncbitaxon\ncbitaxon-term-details.pickle
2023-11-18 11:03:02 INFO [text2term.t2t]: Filtered ontology terms to those of type: class
2023-11-18 11:03:02 INFO [text2term.t2t]: Mapping 1781 source terms to ncbitaxon
2023-11-18 11:06:43 INFO [text2term.t2t]: ...done (mapping time: 220.19s seconds)
5343


In [52]:
t2t_period
print(t2t_period.tail(n=10))

                                  Source Term ID             Source Term  \
5333  http://ccb.hms.harvard.edu/t2t/RGgMK2pniYm              Z. mobilis   
5334  http://ccb.hms.harvard.edu/t2t/RAGfzuYznxQ              Z. tritici   
5335  http://ccb.hms.harvard.edu/t2t/RAGfzuYznxQ              Z. tritici   
5336  http://ccb.hms.harvard.edu/t2t/RAGfzuYznxQ              Z. tritici   
5337  http://ccb.hms.harvard.edu/t2t/R7b9FtTLSvs      Zea mays ssp. mays   
5338  http://ccb.hms.harvard.edu/t2t/R7b9FtTLSvs      Zea mays ssp. mays   
5339  http://ccb.hms.harvard.edu/t2t/R7b9FtTLSvs      Zea mays ssp. mays   
5340  http://ccb.hms.harvard.edu/t2t/R3kh9Pt9xxm  Zea mays ssp. mexicana   
5341  http://ccb.hms.harvard.edu/t2t/R3kh9Pt9xxm  Zea mays ssp. mexicana   
5342  http://ccb.hms.harvard.edu/t2t/R3kh9Pt9xxm  Zea mays ssp. mexicana   

                                    Mapped Term Label  Mapped Term CURIE  \
5333                                 Bacillus mobilis  NCBITAXON:2026190   
5334       

Otherwise, Text2Term performs quite well and the source of error is usually EXTRACT

Many NER tools fail for shorter length terms simply due to increased ambiguation. For EXTRACT, determine if a threshhold number of characters should be used to limit the number of false positives introduced by EXTRACT

Inspect the results of EXTRACT at three and four characters to see if a filter on the Text2Term match score can help limit the false positive rate

In [69]:
%%time
three_letter_list = ['cat','dog','owl','fox','elk','cow','pig','hen','ant','rat','eel','bat','ewe','emu','yak','sow',
                     'CAT','Dog','OWL','Fox','Elk','Cow','Pig','Hen','Ant','Rat','Eel','Bat','Ewe','Emu','Yak','Sow',
                     'eye','hit','man','boy','car','ass','but','why','Moo']
three_letter_match = text2term.map_terms(three_letter_list, "ncbitaxon", use_cache=True)
print(len(three_letter_list),len(three_letter_match))

2023-11-18 12:03:46 INFO [text2term.t2t]: Loading cached ontology from: cache\ncbitaxon\ncbitaxon-term-details.pickle
2023-11-18 12:04:36 INFO [text2term.t2t]: Filtered ontology terms to those of type: class
2023-11-18 12:04:36 INFO [text2term.t2t]: Mapping 41 source terms to ncbitaxon
2023-11-18 12:07:06 INFO [text2term.t2t]: ...done (mapping time: 149.81s seconds)
41 123
CPU times: total: 3min 18s
Wall time: 3min 22s


In [72]:
print(three_letter_match.head(n=2))
three_letter_match.to_csv(os.path.join(output_path,'three_letter_thresh.tsv'), sep='\t', header=True)

                               Source Term ID Source Term Mapped Term Label  \
0  http://ccb.hms.harvard.edu/t2t/R4M5P5PZMSo         cat       Felis catus   
1  http://ccb.hms.harvard.edu/t2t/R4M5P5PZMSo         cat    Catopuma badia   

  Mapped Term CURIE                                 Mapped Term IRI  \
0    NCBITAXON:9685   http://purl.obolibrary.org/obo/NCBITaxon_9685   
1   NCBITAXON:61454  http://purl.obolibrary.org/obo/NCBITaxon_61454   

   Mapping Score  Tags  
0          0.960  None  
1          0.632  None  


In [74]:
%%time
four_letter_list = ['lion','goat','deer','bear','seal','cats','dogs','bats','Rats','Wolf','duck',
                    'Hare','toad','Vole','Crab','clam','kids','lynx','LCMV','MRSA']

four_letter_match = text2term.map_terms(four_letter_list, "ncbitaxon", use_cache=True)
print(len(four_letter_list),len(four_letter_match))

2023-11-20 07:00:12 INFO [text2term.t2t]: Loading cached ontology from: cache\ncbitaxon\ncbitaxon-term-details.pickle
2023-11-20 07:00:52 INFO [text2term.t2t]: Filtered ontology terms to those of type: class
2023-11-20 07:00:52 INFO [text2term.t2t]: Mapping 20 source terms to ncbitaxon
2023-11-20 07:02:49 INFO [text2term.t2t]: ...done (mapping time: 116.83s seconds)
20 60
CPU times: total: 2min 37s
Wall time: 2min 38s


In [75]:
four_letter_match.to_csv(os.path.join(output_path,'four_letter_thresh.tsv'), sep='\t', header=True)

In [91]:
five_letter_matches = unique_t2t_result.loc[unique_t2t_result['original mapped term'].astype(str).str.len()==5]
## There are ~847 rows to review for five letter matches, reduce the number for evaluation
five_letter_sample = random.sample(five_letter_matches['original mapped term'].unique().tolist(),33)
five_letter_subset = five_letter_matches.loc[five_letter_matches['original mapped term'].isin(five_letter_sample)]

print(len(five_letter_subset))
print(five_letter_subset.head(n=5))

99
                                 Source Term ID original mapped term  \
6    http://ccb.hms.harvard.edu/t2t/R4TETiNZMQ7                mouse   
7    http://ccb.hms.harvard.edu/t2t/R4TETiNZMQ7                mouse   
8    http://ccb.hms.harvard.edu/t2t/R4TETiNZMQ7                mouse   
108  http://ccb.hms.harvard.edu/t2t/R7Eh7VS57VF                Yeast   
109  http://ccb.hms.harvard.edu/t2t/R7Eh7VS57VF                Yeast   

             Mapped Term Label Mapped Term CURIE  \
6                 Mus musculus   NCBITAXON:10090   
7                  Mus <genus>   NCBITAXON:10088   
8             Polyplax serrata  NCBITAXON:468196   
108  Saccharomyces pastorianus   NCBITAXON:27292   
109         Pichia sp. Yeast 2  NCBITAXON:553839   

                                     Mapped Term IRI  Original Mapping Score  \
6     http://purl.obolibrary.org/obo/NCBITaxon_10090                   0.999   
7     http://purl.obolibrary.org/obo/NCBITaxon_10088                   0.999   
8    http:/

In [93]:
five_letter_subset.to_csv(os.path.join(output_path,'five_letter_thresh.tsv'), sep='\t', header=True)

## EXTRACT character limit test
Looking at the sampled data for five letters, it looks like Text2Term is doing a good job mapping, but the false positive rate from EXTRACT is affecting the results. Check to see the likelihood of false positives getting dropped AFTER a text2term matching due to low score

In [101]:

ExT2T_stat = {}
three_letter_EXTRACT = unique_t2t_result.loc[unique_t2t_result['original mapped term'].astype(str).str.len()==3]
ExT2T_stat['number of 3 letter terms extracted'] = len(three_letter_EXTRACT['original mapped term'].unique().tolist())
ExT2T_stat['3 letter term list'] = three_letter_EXTRACT['original mapped term'].unique().tolist()
print(len(three_letter_EXTRACT['original mapped term'].unique().tolist()))
print(three_letter_EXTRACT['original mapped term'].unique().tolist())
three_letter_check = text2term.map_terms(three_letter_EXTRACT['original mapped term'].unique().tolist(), "ncbitaxon", use_cache=True)
three_letter_pass = three_letter_check.loc[three_letter_check['Mapping Score']>0.95]
ExT2T_stat['3 letter terms with >0.95 match via T2T'] = three_letter_pass['Source Term'].unique().tolist()
ExT2T_stat['number of 3 letter terms after T2T filter'] = len(three_letter_pass['Source Term'].unique().tolist())
print(len(three_letter_pass['Source Term'].unique().tolist()))


159
['rat', 'HIV', 'pig', 'p22', 'TIV', 'Rat', 'HCV', 'ape', 'dog', 'TCV', 'HeV', 'MLV', 'IAV', 'MYA', 'UCA', 'HBV', 'HPV', 'cow', 'TEV', 'IPs', 'GH1', 'e16', 'RSV', 'osa', 'mcf', 'EBV', 'CMV', 'yak', 'Bcg', 'MAV', 'P22', 'mCG', 'SSP', 'NNV', 'RRV', 'NT1', 'TBV', 'mac', 'EOS', 'rye', 'iDA', 'sIA', 'MTb', 'HEV', 'SAG', 'AKA', 'GAS', 'HOA', 'vir', 'WMV', 'chp', 'JEV', 'Eos', 'NDV', 'ULA', 'e11', 'SFs', 'MCG', 'TSV', 'Doa', 'ATV', 'DWV', 'ALV', 'bee', 'HH1', 'eos', 'AON', 'AMV', 'BLV', 'VZV', 'pax', 'YFV', 'Aon', 'VIR', 'CB1', 'SUS', 'elk', 'Bpv', 'ChP', 'DCV', 'SP5', 'RKN', 'SOa', 'RDV', 'kob', 'Cow', 'SMV', 'e17', 'SV5', 'Sp4', 'Ips', 'Oar', 'MVV', 'ami', 'e19', 'PVY', 'ula', 'PMV', 'DMV', 'pMV', 'Sus', 'AS5', 'IBV', 'SCV', 'ECC', 'ITi', 'UCa', 'DOA', 'AOA', 'cEV', 'Vir', 'HDV', 'mAC', 'mev', 'LMV', 'BDV', 'isa', 'ena', 'SpV', 'TOs', 'Hpb', 'Sag', 'DRV', 'YMW', 'HEA', 'MEV', 'MPV', 'BEV', 'NMV', 'mCF', 'IpA', 'PrV', 'PVX', 'TMV', 'sav', 'FPV', 'IVE', 'FV3', 'BOA', 'QIA', 'hH1', 'UFO', '

In [102]:
print(three_letter_pass['Source Term'].unique().tolist())
## At a match score of > 0.95, the percentage of EXTRACT terms that appear to be true terms is 
## 12 / 48

['rat', 'pig', 'Rat', 'ape', 'dog', 'MYA', 'UCA', 'cow', 'IPs', 'osa', 'yak', 'EOS', 'rye', 'sIA', 'AKA', 'HOA', 'Eos', 'ULA', 'Doa', 'bee', 'eos', 'AON', 'pax', 'Aon', 'elk', 'SOa', 'kob', 'Cow', 'Ips', 'Oar', 'ami', 'ula', 'ITi', 'UCa', 'DOA', 'AOA', 'isa', 'ena', 'HEA', 'IpA', 'IVE', 'UFO', 'ips', 'oar', 'UTA', 'PoA', 'Yak', 'ufo']


In [103]:
four_letter_EXTRACT = unique_t2t_result.loc[unique_t2t_result['original mapped term'].astype(str).str.len()==4]
ExT2T_stat['number of 4 letter terms extracted'] = len(four_letter_EXTRACT['original mapped term'].unique().tolist())
ExT2T_stat['4 letter term list'] = four_letter_EXTRACT['original mapped term'].unique().tolist()
print(len(four_letter_EXTRACT['original mapped term'].unique().tolist()))
four_letter_check = text2term.map_terms(four_letter_EXTRACT['original mapped term'].unique().tolist(), "ncbitaxon", use_cache=True)
four_letter_pass = four_letter_check.loc[four_letter_check['Mapping Score']>0.95]
ExT2T_stat['4 letter terms with >0.95 match via T2T'] = four_letter_pass['Source Term'].unique().tolist()
ExT2T_stat['number of 4 letter terms after T2T filter'] = len(four_letter_pass['Source Term'].unique().tolist())
print(len(four_letter_pass['Source Term'].unique().tolist()))

370
2023-11-20 08:14:23 INFO [text2term.t2t]: Loading cached ontology from: cache\ncbitaxon\ncbitaxon-term-details.pickle
2023-11-20 08:14:54 INFO [text2term.t2t]: Filtered ontology terms to those of type: class
2023-11-20 08:14:54 INFO [text2term.t2t]: Mapping 370 source terms to ncbitaxon
2023-11-20 08:16:41 INFO [text2term.t2t]: ...done (mapping time: 107.13s seconds)
181


In [104]:
print(four_letter_pass['Source Term'].unique().tolist())

['mice', 'Mice', 'H1N1', 'rice', 'flax', 'apes', 'rats', 'dogs', 'inti', 'corn', 'sigA', 'GAGA', 'SAGA', 'goat', 'H3N2', 'Babr', 'aRNA', 'H5N1', 'pupa', 'LIMe', 'betA', 'atlA', 'PUMA', 'MOPS', 'Glia', 'MICA', 'Rats', 'pear', 'bees', 'ARIA', 'IOBA', 'Pima', 'tasA', 'TasA', 'coma', 'aloe', 'topi', 'PilA', 'lice', 'Loma', 'H7N9', 'carp', 'oats', 'Ugni', 'moPs', 'MyLa', 'MACE', 'CreX', 'MOAs', 'Homo', 'SaGA', 'eNOS', 'duck', 'LamA', 'mASO', 'NatA', 'Clea', 'Apis', 'AtlA', 'SigA', 'RANA', 'lion', 'nusA', 'H9N2', 'Bees', 'PISA', 'naso', 'CLEA', 'Goat', 'SIMO', 'ArcA', 'ENOS', 'SigE', 'Lice', 'TAMU', 'THOR', 'MetA', 'PATU', 'RITA', 'netA', 'NetA', 'plum', 'alcA', 'Pupa', 'aroA', 'AroA', 'LAMA', 'MicA', 'nepA', 'disA', 'neem', 'rana', 'MeTA', 'sigE', 'Ucla', 'aliA', 'afer', 'rapa', 'pacu', 'hemp', 'arcA', 'pLUM', 'ANIA', 'aniA', 'deer', 'thor', 'MoAs', 'Gulo', 'nosA', 'rosA', 'Aves', 'EChO', 'beet', 'mino', 'IpsA', 'AphA', 'BIAs', 'nasa', 'DURA', 'katA', 'taro', 'iDAs', 'SutA', 'aria', 'Ulex',

In [108]:
ExT2T_stat['number of 5 letter terms extracted'] = len(five_letter_matches['original mapped term'].unique().tolist())
ExT2T_stat['5 letter term list'] = five_letter_matches['original mapped term'].unique().tolist()
print(len(five_letter_matches['original mapped term'].unique().tolist()))
five_letter_pass = five_letter_matches[five_letter_matches['Original Mapping Score']>0.95]
ExT2T_stat['5 letter terms with >0.95 match via T2T'] = five_letter_pass['Source Term'].unique().tolist()
ExT2T_stat['number of 5 letter terms after T2T filter'] = len(five_letter_pass['Source Term'].unique().tolist())
print(len(five_letter_pass['Source Term'].unique().tolist()))

283
127


In [114]:
five_pass = five_letter_pass['Source Term'].unique().tolist()
print(five_pass)
with open(os.path.join(output_path,'five_letter_pass.tsv'),'w') as outlist:
    for eachword in five_pass:
        outlist.write(eachword+'\n')

['human', 'mouse', 'swine', 'flies', 'birds', 'nanos', 'wheat', 'maize', 'venus', 'goats', 'lamia', 'lambs', 'sheep', 'fungi', 'apple', 'valsa', 'helix', 'frogs', 'smaug', 'codon', 'cocoa', 'galea', 'wasps', 'turbo', 'alexa', 'aroma', 'horse', 'hydra', 'ticks', 'areca', 'cacao', 'lemon', 'danio', 'mucor', 'bream', 'hippa', 'phoma', 'panda', 'ramie', 'ciona', 'paris', 'radix', 'boars', 'goose', 'geese', 'acari', 'malus', 'peach', 'thada', 'triso', 'gemma', 'acris', 'moths', 'janus', 'aegis', 'onion', 'equus', 'krill', 'aedes', 'culex', 'disco', 'geron', 'nihon', 'pinna', 'arida', 'tiger', 'agria', 'psara', 'anise', 'thyme', 'dingo', 'aotus', 'gadus', 'dorea', 'tampa', 'apela', 'eland', 'ducks', 'bears', 'vitis', 'tulsa', 'monza', 'bursa', 'oryza', 'taxus', 'morel', 'mixta', 'babax', 'pears', 'seals', 'fleas', 'arima', 'camel', 'theba', 'fagus', 'vinca', 'clove', 'lotus', 'ixora', 'midge', 'gesta', 'seila', 'palea', 'peria', 'delta', 'xenia', 'llama', 'lethe', 'pecan', 'perro', 'cornu', 

In [115]:
ExT2T_stat['3 letter false positives after filter'] = 48-12
ExT2T_stat['4 letter false positives after filter'] = 181-42
ExT2T_stat['5 letter false positives after filter'] = 36
ExT2T_stat['5 letter true positives that are standalone terms'] = 47
ExT2T_stat['5 letter true positives that are NOT standalone terms (part of another term)'] = 41
with open(os.path.join(output_path,'t2t_EXTRACT_filter_test.json'),'w') as outwrite:
    outwrite.write(json.dumps(ExT2T_stat))

# Heuristics for improving accuracy based on the results of the EXTRACT text mapping

1. Text2Term is weaker at mapping terms with fewer letters so a score cut off should be applied for terms with 3 letters, 4 letters, and 5 letters
    * For 3 letter terms
      * The capitalization will not affect the match terms
      * A match score of >0.95 is likely to be correct
      * However, the number of extracted terms that are likely to be true terms vs false positives is only 12/48 even after meeting this threshhold. For this reason, we should exclude terms of 3 or less characters as the percentage EXTRACTed correctly is pretty low
    * For 4 letter terms
      * At a score of >0.95, cats, rats and bats will be false negatives (matched well, but dropped), while duck (matched a little too specifically to domestic duck) will pass
      * At a score of >0.91, cats, rats, and bats will pass, duck will still be an issue
      * However, the number of extracted terms that are likely to be true terms is only ~42/181. For this reason, we should also exclude terms of 4 or less characters
    * For 5 letter terms
      * At a score of >0.95, the number of true positives is 91/127
      * Of those 91 true positive terms, about 41 are terms which are part of a taxonomic phrase (i.e. - either only the genus part of a taxonomy or a species part
      * This means that only about 41 of these terms would be missed with the number of letters threshhold for EXTRACT were >5, the remaining true positives would likely be captured with the whole term
2. Text2Term is also weaker at mapping terms formatted as g. species, as it can score higher mapping to the correct species term but incorrect genus. To address this:
    * Identify such terms using regex (r"\b[A-Z]\.\s[^\s]+\b")
    * For these terms, only take the result if the genus letter matches the first letter of the mapped result (i.e. split on '.', take first)
    * There will be plenty of exceptions, but this should address the majority of problematic matches